# Chapter 10 Cython Numpy and Typed Memoryviews

### I. The Power of the New Buffer Protocol
The new buffer protocol is a C-level protocol。Python对象可以实现该协议而不影响它们Python-level的交互。该协议定义了一个C-level struct，其有一个data buffer，以及metadata用来描述buffer的layout、数据类型、读写权限，还定义了协议要求必须实现的API。

new buffer protocol最重要的特征是其以不同的表达方式表示相同底层数据的能力。它允许Numpy、集中Python内建类型、Cython-level array-like对象无需复制即可共享相同的数据。以下是实现了该协议的一些类型：
- *Numpy* <font face='consolas'>ndarray</font>
- *Build-in* <font face='consolas'>str</font>:只有Python 2.6和2.7的内建字符串类型实现了该协议。Python 2的Unicode类型以及Python 3的string类型则没有实现。
- *Build-in* <font face='consolas'>bytes *and* bytearray</font>
- *Standard library* <font face='consolas'>array.array</font>
- *Standard library* ctypes<font face='consolas'>arrays</font>
- *Various third-party types* : 譬如PIL为很多图像类型实现了该协议

#### The memoryview Type
Python内建的<font face='consolas'>memoryview</font>类型的唯一目的就是在Python-level展示C-level buffer。创建<font face='consolas'>memoryview</font>的方法就是向构造器传入实现了该协议的对象。

In [20]:
bb = b"These are the times that try men's souls."
ba = bytearray(bb)
memv = memoryview(bb)
mutable1 = memoryview(ba)
print memv
# 通过index访问底层buffer的相应数据
print memv[3]
# memoryview的切片也是一个memoryview
print memv[3:10]
print memv[3:10][0]
# 获取metadata readonly，查看是否只读。
print memv.readonly
# 通过memoryview改变非只读memory的值
mutable1[0]='F'
mutable1[0],ba

s
s
True


('F', bytearray(b"Fhese are the times that try men\'s souls."))

In [19]:
import numpy as np
np_mv = memoryview(np.ones((10,20,30)))
np_mv_F = memoryview(np.ones((10,20,30),order='F'))
print np_mv.shape
# np_mv.strides显示的每一维度相临两元素相差的字节数。
# 可以看到C风格从最后一维开始填充数据，而F风格从第一维开始填充
print 'C order:',np_mv.strides
print 'F order:',np_mv_F.strides
# 底层数据的类型
print np_mv.format 

(10L, 20L, 30L)
C order: (4800L, 240L, 8L)
F order: (8L, 80L, 1600L)
d


### II. Typed Memoryviews
Cython有名为*typed memoryview*的C-level类型，和<font face='consolas'>memoryview</font>概念重叠并有一定扩展。由于typed momoryview在C级别操作，其具有最小的Python开销，所以非常高效；其具有和memoryview相似的接口，所以比直接操作C-level buffer要简单的多。

#### Typed Memoryview Example
- 声明<font face='consolas'>typed memoryview</font>类型$\Longrightarrow$
<font face='consolas'>double[:] mv</font>
    - <font face='consolas'>double</font>: 类型
    - <font face='consolas'>[:]</font>: 一个冒号说明是一维memoryview对象
    - <font face='consolas'>mv</font>: 名称
- 在下面的例子中，遍历一个typed memoryview，Cython本质上时将它当做一个general Python iterator，遍历中每次对数据的访问都会调用Python/C API，所以可以看到代码的效率比直接使用Python实现还要低。

In [29]:
%%file memviews.pyx

def summer(double[:] mv):
    '''Sum its argument's contents.'''
    cdef double d, ss = 0.0
    for d in mv:
        ss += d
    return ss

Overwriting memviews.pyx


In [30]:
import pyximport
pyximport.install()
import memviews
import numpy as np
arr = np.ones((10**6,), dtype=np.double)
def summer(mv):
    '''Sum its argument's contents.'''
    ss=0.
    for d in mv:
        ss += d
    return ss
# np.ndarray对象赋给memoryview参数，由于np.ndarray实现了buffer
# 所以mv可以访问np.ndarray底层的数据。如果传入的参数没有实现buffer
# 则会报出ValueError。
%timeit memviews.summer(arr)
%timeit summer(arr)

1 loops, best of 3: 196 ms per loop
10 loops, best of 3: 95 ms per loop


#### C-Level Access to Typed Memoryview Data
当我们使用一个静态定义的整数索引一个typed memoryview时，Cython会生成代码规避Python/C API的调用，而直接索引底层的fuffer，这可以大幅提升效率。
> When indexing into a typed memoryview with a typed integer, Cython generates code that bypass Python/C API calls and indexes into the underlying buffer directly.

#### Trading Safety for Performance
每次我们访问memoryview，Cython都会检查index是否超出范围。如果超出范围，Cython会报出IndexError。与Python类似，Cython也允许我们使用负索引（index wraparound）。

当我们知道我们永远不会超出范围或使用负索引，我们可以通过将cython模块中的boundscheck和wraparound编译指令设为False，关闭这些检查，提高性能:

- 关闭某段代码的boundscheck和wraparoundcheck
<font face='consolas'>from cython cimport boundscheck，wraparound</font><br></br>
<font face='consolas'>......</font><br></br>
<font face='consolas'>with boundscheck(False),wraparound(False):</font>

- 要关闭整个函数的boundscheck和wraparoundcheck，我们需要删除context manager形式（with语句）的编译指令，改用装饰器形式的指令。
<font face='consolas'>@boundscheck(False)</font><br></br>
<font face='consolas'>@wraparound(False)</font>

- 要关闭整个扩展模块的boundscheck和wraparoundcheck，我们在文件顶部使用注释形式的编译指令。
<font face='consolas'># cython: boundscheck=False</font><br></br>
<font face='consolas'># cython: wraparoundcheck=False</font>

In [51]:
%%file memviews_ii.pyx
# 使用一个静态定义的整数索引一个typed memoryview
from cython cimport boundscheck, wraparound
cpdef double summer(double[:] mv):
    '''Sum its argument's contents.'''
    cdef:
        double ss = 0.0
        int i, N
    N = mv.shape[0]
    with boundscheck(False), wraparound(False):
        for i in range(N):
            ss += mv[i]
    return ss

Overwriting memviews_ii.pyx


In [53]:
import pyximport
pyximport.install()
import memviews_ii
import numpy as np
arr = np.ones((10**6,), dtype=np.double)
%timeit memviews_ii.summer(arr)

1000 loops, best of 3: 1.3 ms per loop


In [54]:
!cython -a memviews.pyx
!cython -a memviews_ii.pyx

#### Declaring Typed Memoryviews
定义typed memoryviews时，我们可以控制很多属性：
- *Element type*<br></br>
typed memoryview的元素类型可以是数值标量，如<font face='consolas'>int, float, double complex, ctypedef别名, cdef struct定义的结构化类型, fused types(开发中)。</font>
- *Dimensionality*<br></br>
目前typed memoryview最多支持7维。要声明一个三维typed memoryview，在元素类型后使用<font color='#ff2222' face='consolas'>[:,:,:]</font>
- *Contiguous or strided data packing*<br></br>
typed memoryview的strided dimension（以一个冒号声明）和一个strided（非连续，但是步距规律）buffer维度兼容。默认strided access。<font color='#ff2222'>？？不是很明白</font>
- *C or Fortran contiguity*<br></br>
    - C风格(column major)：buffer作为一个整体在内存中时连续的，但是多维化后其<font color='#ff2222'>最后一维</font>是连续的。
    - F风格(raw major)：buffer作为一个整体在内存中时连续的，但是多维化后其<font color='#ff2222'>第一维</font>是连续的。
- *Direct or indirect access*<br></br>
默认直接访问，这覆盖了几乎所有的使用情况——其指明这个维度可以使用直接的索引算术（indexing arithmetic）来直接访问底层的数据。如果指明一个维度是间接的，那么底层的buffer保存了剩余array的指针，在访问的时候必须被dereferenced。Numpy只支持直接访问。<font color='#ff2222'>？？？？？完全不明白</font>

声明C风格memoryview：最后一个维度为"<font color='#ff2222' face='consolas'>::1</font>"，表示最后一个维度stride只能为1（及内存中相邻）。此时如果传入的ndarray最后一个维度stride不为1，会报出ValueError。<br></br>
声明F风格memoryview：第一个维度为"<font color='#ff2222' face='consolas'>::1</font>"，表示第一个维度stride只能为1（及内存中相邻）。<br></br>
fully strided、C风格、F风格的typed memoryview声明覆盖了大部分的情况。根据不同的问题选用合适的类型。

<font color='#ff2222' face='consolas'>Numpy提供了ascontiguousarray和asfortranarray</font>转换函数，以一个array-like对象为参数，返回一个保证具有C或Fortran风格连续性的Numpyarray。

In [ ]:
# default typed memoryview declaration for a three-dimensional object:
cdef int[:, :, :] mv
mv = np.empty((10,20,30), dtype=np.int32)
# 从Fortran风格排列的ndarray获取buffer
mv_F = np.ones((10,20,30), dtype=np.int32, order='F')
# 从fully strided ndarray获取buffer
arr = np.ones((13,17,19), dtype=np.int32)
mv_strided = arr[4:10:2, ::3, 5::-2]

In [57]:
%%file memviews_fused.pyx
# Cython的fused type可以作为typed memoryview类型
# 导入fused type后，其至少要用来声明一个变量，这样Cython能够决定
# 在编译时或运行时到底使用fused type中的那个子类型。
from cython cimport floating
from cython cimport boundscheck, wraparound
cpdef floating generic_summer(floating[:] mv):
    '''Sum its argument's contents.'''
    cdef:
        floating ss = 0.0
        int i, N
    N = mv.shape[0]
    with boundscheck(False), wraparound(False):
        for i in range(N):
            ss += mv[i]
    return ss

Overwriting memviews_fused.pyx


In [60]:
import pyximport
pyximport.install()
import numpy as np
double_array = np.arange(10., dtype=np.double)
float_array = np.asarray(double_array, dtype=np.float)
from memviews_fused import generic_summer

In [61]:
print generic_summer(double_array)
#=> 1000000.0
print generic_summer(float_array)
#=> 1000000.0

45.0
45.0


#### Using Typed Memoryviews
一旦我们声明了一个typed memoryview，我们必须将一个支持buffer的对象赋给它。这样这个typed memoryview可以获得（视图）右侧对象的buffer。

typed memoryview支持的操作（与Numpy类似）：
- 和numpy相似的索引访问、切片访问、修改底层的对应数据。
    - 当我们要将数据复制到一个typed memoryview，一定要使用切片赋值。
- 使用<font face='consolas'>copy或copy_fortran</font>方法生成memoryview对应buffer的C或Fortran风格的副本。
- 可以对memoryview切片得到另一个typed memoryview，两者共享内存。
- 可以通过None在typed memoryview中插入新的维度。page203

与Numpy不同的是typed memoryview<font color='#ff2222'>不支持universal function</font>。可以将typed memoryview转化为numpy，在用universal function进行操作。

最后，C-contiguous typed memoryview的转置是一个Fortran-contiguous typed memoryview

In [62]:
%%file UseTypedMemoryview.pyx
import numpy as np
import math

cdef int[:,:] mv = np.ones((20,30))
print mv[10,-20] # 访问底层buffer的对应数据
mv[0,-1] = 3 # 修改底层buffer的对应数据

cdef double[:,:] mv = np.empty((10,20))
mv[...] = math.pi # ...表示将整个typed memoryview赋为math.pi

# 可以将维度、数据类型相同的一个typed memoryview底层buffer数据
# 复制给另一个typed memoryview
cdef double[:, :] mv1 = np.zeros((10, 20))
cdef double[:, ::1] mv2 = np.ones((20, 40))
mv1[::2, ::2] = mv2[1:11:2, 10:40:3]

# 通过None在typed memoryview中插入新的维度
cdef double[:] mv = np.ones((50,))
assert mv[None, :].shape == (1, 50)
assert mv[:, None].shape == (50, 1)

# typed memoryview不支持universal function
# 所以也不支持broadcasting
# 可以将其转化为ndarray在进行操作
cdef float[:] rows = np.arange((100,), dtype=np.float32)
cdef float[:] cols = rows
# broadcasting sum
plane = np.asarray(rows[:,None]) + np.asarray(cols[None,:])

# C-contiguous typed memoryview转置后是一个
# Fortran-contiguous typed memoryview
cdef int[:, ::1] c_contig = obj
cdef int[::1, :] f_contig = c_contig.T

Writing UseTypedMemoryview.pyx


#### Beyond Buffers
除了Python-space objects，typed memoryview开可以用于C-level array：[1]动态堆分配数组，或[2]固定长度栈数组